In [52]:
import os
from dotenv import load_dotenv
load_dotenv()

# Load our OpenAI api key from the env file
OPENAI_KEY = os.getenv('OPENAI_KEY')

In [53]:
from langchain.llms import OpenAI

# Initialize our model with 0.6 temperature
llm = OpenAI(temperature=0.6, api_key=OPENAI_KEY)

# Ask a question to the model, and store the response in 'name'
name = llm("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
print(name)



"Maharaja's Palace" 


In [55]:
# We dont want to change our prompt everytime for different restaurant themes
# To solve this, we will create a 'Prompt Template'
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

prompt_template_name.format(cuisine="Italian") 

'I want to open a restaurant for Italian food. Suggest a fancy name for this.'

In [56]:
# Now we will create a chain, which will enable us to use our model with the template we created
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("American")

'\n\n"The Stateside Bistro"'

In [57]:
# What if we want to create a restaurant menu from the restaurant name generated by the model?
# We will use 'Simple Sequential Chain' for this. 

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """Give me a comma seperated text that has menu items for the restaurant named {restaurant_name}."""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [63]:
from langchain.chains import SimpleSequentialChain

# Create Simple Sequential Chain
chain = SimpleSequentialChain(chains= [name_chain, food_items_chain])
response = chain.run("American")
print(response)



"Smoked BBQ Ribs, Fried Chicken Platter, Pulled Pork Sandwich, Mac and Cheese, Collard Greens, Fried Green Tomatoes, Shrimp Po' Boy, Cajun Jambalaya, Fried Catfish, Chicken and Waffles"


In [60]:
# With Simple Sequential Chain we only get 1 output, which is the menu items list. But we want to get the restaurant name too.
# We will use Sequential Chain for this.
llm = OpenAI(temperature=0.7, api_key=OPENAI_KEY)

# Chain 1: Restaurant Name
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

# Chain 2: Menu Items
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """Give me a comma seperated text that has menu items for the restaurant named {restaurant_name}."""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [62]:
from langchain.chains import SequentialChain

# Create Sequential Chain
chain = SequentialChain(
    chains= [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

chain({'cuisine':'Italian'})

{'cuisine': 'Italian',
 'restaurant_name': '\n\n"Bella Cucina Trattoria"',
 'menu_items': '\n\n"Antipasto, Insalata Caprese, Bruschetta, Gnocchi, Lasagna, Fettuccine Alfredo, Chicken Parmigiana, Veal Marsala, Shrimp Scampi, Grilled Salmon, Tiramisu, Cannoli"'}